In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00


In [2]:
!pip install transformers

In [3]:
!pip install scikit-multilearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 971.3 kB/s eta 0:00:00


In [4]:
# !nvidia-smi

In [5]:
import pandas as pd

from transformers import BertForSequenceClassification, BertTokenizer, BertConfig, BertModel
from transformers import RobertaForSequenceClassification, RobertaTokenizer, RobertaConfig

from skmultilearn.model_selection import iterative_train_test_split

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics.cluster import contingency_matrix

import shutil
import os
import torch
import json
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
np.random.seed(42)


# Model helper functions and classes

In [6]:
def pre_process_data(df, test_proportion, train_size):

    df.columns.values[0] = "FQText"
    size_df = df.shape[0]
    df = shuffle(df, random_state = 42)

    X = df[["FQText"]].to_numpy()

    y_df = df.drop(["FQText"], axis=1).astype(np.float32)
    cols = df.columns
    df_labels = list(y_df.columns)

    y = y_df.to_numpy()
    # print(y_df.info())

    test_proportion = 0.05
    train_size = 0.9

    X_model_train, y_model_train, X_test, y_test = iterative_train_test_split(X, y, test_size=test_proportion)
    X_train, y_train, X_val, y_val = iterative_train_test_split(X_model_train, y_model_train, test_size= 1-train_size)

    df_train = pd.concat([pd.DataFrame(X_train, columns=["FQText"]), pd.DataFrame(y_train, columns=[df_labels], dtype=np.float32)], axis=1).reset_index(drop=True)
    df_val = pd.concat([pd.DataFrame(X_val, columns=["FQText"]), pd.DataFrame(y_val, columns=[df_labels], dtype=np.float32)], axis=1).reset_index(drop=True)
    df_test = pd.concat([pd.DataFrame(X_test, columns=["FQText"]), pd.DataFrame(y_test, columns=[df_labels], dtype=np.float32)], axis=1).reset_index(drop=True)

    df_train.columns = cols
    df_val.columns = cols
    df_test.columns = cols

    # print("Total amount of data: {}".format(size_df))
    # print("Number of rows used to TRAIN: {}".format(df_train.shape[0]))
    # print("Number of rows used to VALIDATE: {}".format(df_val.shape[0]))
    # print("Number of rows used to TEST: {}".format(df_test.shape[0]))

    return df_train, df_val, df_test, df_labels

In [8]:
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['FQText']
        # self.labels = list(df.columns)[2:] # list of the target values
        self.targets = self.df[df_labels].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True, # do we actually need special tokens ??
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'  # pytorch tensors
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }

In [9]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        # self.bert_model = LongformerModel.from_pretrained('allenai/longformer-base-4096', return_dict=True, problem_type="multi_label_classification")
        self.bert_model = BertModel.from_pretrained("bert-base-uncased", return_dict=True, problem_type="multi_label_classification")

        self.dropout = torch.nn.Dropout(0.3) 
        self.linear = torch.nn.Linear(768, len(df_labels)) # have to change he n of possible labels here

    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids,
            attention_mask=attn_mask,
            token_type_ids=token_type_ids
        )

        output_dropout = self.dropout(output.pooler_output)
        # print(output_dropout)
        output = self.linear(output_dropout)
        return output

In [10]:
def loss_fn(outputs, targets):
    # print(outputs, targets)
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

## Train model function

In [11]:
def train_model(n_epochs, training_loader, validation_loader, model,
                optimizer, checkpoint_path, best_model_path, df_labels, test_loader):
  valid_loss_min = np.Inf
  best_metrics = {}

  best_metrics[f"n_epochs"] = n_epochs



  for epoch in range(1, n_epochs+1):

    epoch_n = "epoch_{}".format(epoch - 1)
    best_metrics[epoch_n] = {}

    train_loss = 0
    valid_loss = 0

    model.train()
    print('Epoch {} Training Start'.format(epoch))
    for batch_idx, data in enumerate(training_loader):

        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        """targets are the y array of the original data.
        """

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))




    print('Epoch {} Training End'.format(epoch))

    print('Epoch {} Validation'.format(epoch))
    
 # VALIDATION
    model.eval()

    val_targets = []
    val_outputs = []

    proba_outputs = []

    sigmoid = torch.nn.Sigmoid()

    with torch.no_grad():
      for batch_idx, data in enumerate(validation_loader, 0):
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)
        outputs = model(ids, mask, token_type_ids)

        loss = loss_fn(outputs, targets)
        valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
        val_targets.extend(targets.cpu().detach().numpy().tolist())
        val_outputs.extend(outputs.cpu().detach().numpy().tolist())

        probs = sigmoid(torch.Tensor(outputs))
        proba_outputs.append(probs.cpu().detach().numpy().tolist())

      print('Epoch {}  Metrics\n\n'.format(epoch))
      metrics, f1, roc_auc, hamming, accuracy = multi_labels_metrics(val_outputs, val_targets, df_labels, threshold=0.5)

      print('Epoch {} Validation End'.format(epoch))

      train_loss = train_loss/len(training_loader)
      valid_loss = valid_loss/len(validation_loader)


      best_metrics[epoch_n][f"train_loss_epoch_{epoch}"] = train_loss
      best_metrics[epoch_n][f"valid_loss_epoch_{epoch}"] = valid_loss
      best_metrics[epoch_n][f"f1_epoch_{epoch}"] = f1
      best_metrics[epoch_n][f"accuracy_epoch_{epoch}"] = accuracy
      best_metrics[epoch_n][f"roc_auc_epoch_{epoch}"] = roc_auc
      best_metrics[epoch_n][f"hamming_epoch_{epoch}"] = hamming
      best_metrics[epoch_n][f"proba_outputs_epoch_{epoch}"] = proba_outputs
      best_metrics[epoch_n][f"actual_classes_epoch_{epoch}"] = val_targets


      print('Epoch: {} Training Loss: {:.6f} Validation Loss: {:.6f}'.format(
            epoch,
            train_loss,
            valid_loss
            ))

      if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))

        valid_loss_min = valid_loss
        best_metrics["best_metrics"] = {
                'epoch': epoch,
                'train_loss': train_loss,
                'valid_loss': valid_loss,
                'val_acc': accuracy,
                'val_f1': f1,
                'val_roc_auc': roc_auc,
                'val_hamming': hamming,
                'proba_outputs': proba_outputs,
                'actual_classes': val_targets
            }



    print('Epoch {}  Finished\n'.format(epoch))

  metrics, f1, roc_auc, hamming, accuracy, test_loss, metrics_proba, metrics_targets = compute_test_metrics(test_loader, model)
  best_metrics["test"] = metrics
  best_metrics["labels"] = df_labels

  return model, best_metrics

In [12]:
import numpy as np
from sklearn.metrics import roc_auc_score, f1_score, hamming_loss, accuracy_score

def multi_labels_metrics(predictions, targets, df_labels, threshold=0.36): # mess with threshold

  sigmoid = torch.nn.Sigmoid()
  probs = sigmoid(torch.Tensor(predictions)) #mapply activation function in the raw values

  y_pred = np.zeros(probs.shape)
  y_pred[np.where(probs>=threshold)] = 1
  y_true = targets

  accuracy = accuracy_score(y_true, y_pred)
  f1 = f1_score(y_true, y_pred, average = 'micro')
  roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
  hamming = hamming_loss(y_true, y_pred)
  class_report = classification_report(y_true, y_pred, target_names=df_labels)

  metrics = {
      "roc_auc": roc_auc,      
      "hamming_loss": hamming,  # special multilabel metrics
      "f1": f1,
      "accuracy_score": accuracy
  }
  print(metrics)
  print(class_report)
  return metrics, f1, roc_auc, hamming, accuracy

def compute_test_metrics(test_loader, model):

  model.eval()

  test_loss = 0
  metrics_targets = []
  metrics_outputs = []
  metrics_proba = []

  sigmoid = torch.nn.Sigmoid()
  with torch.no_grad():
        for _, data in enumerate(test_loader, 0):
              ids = data['input_ids'].to(device, dtype = torch.long)
              mask = data['attention_mask'].to(device, dtype = torch.long)
              token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
              targets = data['targets'].to(device, dtype = torch.float)

              outputs = model(ids.squeeze(), mask.squeeze(), token_type_ids.squeeze())

              loss = loss_fn(outputs, targets)
              test_loss = test_loss + ((1 / (_ + 1)) * (loss.item() - test_loss))

              metrics_targets.extend(targets.cpu().detach().numpy().tolist())
              metrics_outputs.extend(outputs.cpu().detach().numpy().tolist())


              probs = sigmoid(torch.Tensor(outputs))

              # val_targets_study.append(targets.cpu().detach().numpy().tolist())
              metrics_proba.append(probs.cpu().detach().numpy().tolist())

  test_loss = test_loss/len(test_loader)

  metrics, f1, roc_auc, hamming, accuracy = multi_labels_metrics(metrics_outputs, metrics_targets, df_labels)
  metrics["test_loss"] = test_loss
  metrics["f1"] = f1
  metrics["roc_auc"] = roc_auc
  metrics["hamming"] = hamming
  metrics["accuracy"] = accuracy

  metrics["test_proba"] = metrics_proba
  metrics["test_targets"] = metrics_targets


  return metrics, f1, roc_auc, hamming, accuracy, test_loss, metrics_proba, metrics_targets


In [13]:
results = []

TRAIN AND VALIDATION LOOP

In [14]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [15]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [16]:
def train_final_models(param_grid, train_df, val_df, df_test, df_labels, n_grids=1):
  i = 0
  grid_metric_results = []


  MAX_LEN, TRAIN_BATCH_SIZE, VALID_BATCH_SIZE, EPOCHS, LEARNING_RATE = set_hyperparams(param_grid)

  train_dataset = CustomDataset(train_df, tokenizer, MAX_LEN)
  valid_dataset = CustomDataset(val_df, tokenizer, MAX_LEN)
  test_dataset = CustomDataset(df_test, tokenizer, MAX_LEN)

  train_data_loader = torch.utils.data.DataLoader(train_dataset,
      batch_size=TRAIN_BATCH_SIZE,
      shuffle=True,
      num_workers=0
  )

  val_data_loader = torch.utils.data.DataLoader(valid_dataset,
      batch_size=VALID_BATCH_SIZE,
      shuffle=False,
      num_workers=0
  )

  test_data_loader = torch.utils.data.DataLoader(test_dataset,
      batch_size=VALID_BATCH_SIZE,
      shuffle=False,
      num_workers=0
  )



  model = BERTClass()
  model.to(device)
  optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

#   for i, left in enumerate(train_data_loader):
#     print(i)
#     with torch.no_grad():
#         temp = model(left).view(-1, 1, 300, 300)
#         right.append(temp.to('cpu'))
#         del temp
#         torch.cuda.empty_cache()

  ckpt_path = "curr_ckpt"
  best_model_path = "best_model.pt"

  trained_model, best_metrics = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, ckpt_path, best_model_path, df_labels, test_data_loader)

  result = {
        'hyperparameters': param_grid,
        'metrics': best_metrics
    }

  grid_metric_results.append(result)
  i = i + 1

  return grid_metric_results

In [17]:
def set_hyperparams(param_grid):
    MAX_LEN = param_grid['max-len']
    TRAIN_BATCH_SIZE = param_grid['batch_size']
    VALID_BATCH_SIZE = param_grid['batch_size']
    EPOCHS = param_grid['num_train_epochs']
    LEARNING_RATE = param_grid['learning_rate']
    return MAX_LEN, TRAIN_BATCH_SIZE, VALID_BATCH_SIZE, EPOCHS, LEARNING_RATE

In [18]:
dfs = ['nancy_determinants_individual_labels_eng.csv',
 'nancy_contents_individual_labels_eng.csv',
 'nancy_contents_macro_labels_english.csv',
 'nancy_determinants_macro_labels_english.csv']

# Individual Determinants

In [24]:
param_grid_ind_dets = {
                        "max-len" : 128,
                        'learning_rate': 3e-5,
                        'batch_size': 8,
                        'num_train_epochs': 19
                                                }


In [25]:
df_name = dfs[0]
data_model = df_name.split("_")
data_model = "_".join(data_model[1:3]) #getting the name to name the different dictionaries
df = pd.read_csv(df_name)

test_proportion = 0.05
train_size = 0.9

train_df, val_df, df_test, df_labels  = pre_process_data(df, test_proportion, train_size)

In [21]:
print(df_name)

nancy_determinants_individual_labels_eng.csv


In [26]:
results_individual_determinant_final = train_final_models(param_grid_ind_dets, train_df, val_df, df_test, df_labels)

############# Epoch 1: Training Start   #############
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1  Metrics   #############


{'roc_auc': 0.5, 'hamming_loss': 0.06481481481481481, 'f1': 0.0, 'accuracy_score': 0.0}
              precision    recall  f1-score   support

           C       0.00      0.00      0.00         3
          C'       0.00      0.00      0.00         2
         C'F       0.00      0.00      0.00         1
          CF       0.00      0.00      0.00         1
         CF'       0.00      0.00      0.00         0
        CLOB       0.00      0.00      0.00         1
       CLOBF       0.00      0.00      0.00         0
           E       0.00      0.00      0.00         3
          EF       0.00      0.00      0.00         1
           F       0.00      0.00      0.00        14
          FC       0.00      0.00      0.00         3
         FC'       0.00      0.00      0.00       

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344:

############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2  Metrics   #############


{'roc_auc': 0.5, 'hamming_loss': 0.06481481481481481, 'f1': 0.0, 'accuracy_score': 0.0}
              precision    recall  f1-score   support

           C       0.00      0.00      0.00         3
          C'       0.00      0.00      0.00         2
         C'F       0.00      0.00      0.00         1
          CF       0.00      0.00      0.00         1
         CF'       0.00      0.00      0.00         0
        CLOB       0.00      0.00      0.00         1
       CLOBF       0.00      0.00      0.00         0
           E       0.00      0.00      0.00         3
          EF       0.00      0.00      0.00         1
           F       0.00      0.00      0.00        14
          FC       0.00      0.00      0.00         3
         FC'       0.00      0.00      0.00         2
       FCLOB       0.00      0.00      0.00       

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344:

############# Epoch 3: Training End     #############
############# Epoch 3: Validation Start   #############
############# Epoch 3  Metrics   #############


{'roc_auc': 0.5673031588873173, 'hamming_loss': 0.06327160493827161, 'f1': 0.22641509433962262, 'accuracy_score': 0.16666666666666666}
              precision    recall  f1-score   support

           C       0.00      0.00      0.00         3
          C'       0.00      0.00      0.00         2
         C'F       0.00      0.00      0.00         1
          CF       0.00      0.00      0.00         1
         CF'       0.00      0.00      0.00         0
        CLOB       0.00      0.00      0.00         1
       CLOBF       0.00      0.00      0.00         0
           E       0.00      0.00      0.00         3
          EF       0.00      0.00      0.00         1
           F       0.55      0.43      0.48        14
          FC       0.00      0.00      0.00         3
         FC'       0.00      0.00      0.00         2
   

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


############# Epoch 17: Training End     #############
############# Epoch 17: Validation Start   #############
############# Epoch 17  Metrics   #############


{'roc_auc': 0.6999057048561999, 'hamming_loss': 0.043209876543209874, 'f1': 0.5483870967741935, 'accuracy_score': 0.3611111111111111}
              precision    recall  f1-score   support

           C       0.00      0.00      0.00         3
          C'       1.00      0.50      0.67         2
         C'F       0.00      0.00      0.00         1
          CF       0.00      0.00      0.00         1
         CF'       0.00      0.00      0.00         0
        CLOB       0.00      0.00      0.00         1
       CLOBF       0.00      0.00      0.00         0
           E       1.00      0.33      0.50         3
          EF       0.00      0.00      0.00         1
           F       0.83      0.71      0.77        14
          FC       0.00      0.00      0.00         3
         FC'       0.00      0.00      0.00         2
 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


############# Epoch 18: Training End     #############
############# Epoch 18: Validation Start   #############
############# Epoch 18  Metrics   #############


{'roc_auc': 0.7109853842527111, 'hamming_loss': 0.043209876543209874, 'f1': 0.5625, 'accuracy_score': 0.3888888888888889}
              precision    recall  f1-score   support

           C       0.00      0.00      0.00         3
          C'       1.00      0.50      0.67         2
         C'F       0.00      0.00      0.00         1
          CF       0.00      0.00      0.00         1
         CF'       0.00      0.00      0.00         0
        CLOB       0.00      0.00      0.00         1
       CLOBF       0.00      0.00      0.00         0
           E       1.00      0.33      0.50         3
          EF       0.00      0.00      0.00         1
           F       0.79      0.79      0.79        14
          FC       0.00      0.00      0.00         3
         FC'       0.00      0.00      0.00         2
       FCLOB 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [23]:
with open("metrics_final_individual_determinant.json", "w") as f:
    json.dump(results_individual_determinant_final, f)

# Individual Contents

In [39]:
param_grid_ind_conts = {
                        'max-len' : 128,
                        'learning_rate': 3e-5,
                        'batch_size': 8,
                        'num_train_epochs': 32
                                                }


In [40]:
df_name = dfs[1]
data_model = df_name.split("_")
data_model = "_".join(data_model[1:3]) #getting the name to name the different dictionaries
df = pd.read_csv(df_name)

test_proportion = 0.05
train_size = 0.9

train_df, val_df, df_test, df_labels  = pre_process_data(df, test_proportion, train_size)

In [41]:
print(df_name)

nancy_contents_individual_labels_eng.csv


In [42]:
results_individual_determinant_final = train_final_models(param_grid_ind_conts, train_df, val_df, df_test, df_labels)

############# Epoch 1: Training Start   #############
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1  Metrics   #############


{'roc_auc': 0.5, 'hamming_loss': 0.04245709123757904, 'f1': 0.0, 'accuracy_score': 0.0}
              precision    recall  f1-score   support

         (A)       0.00      0.00      0.00         1
        (AD)       0.00      0.00      0.00         0
         (H)       0.00      0.00      0.00         2
        (HD)       0.00      0.00      0.00         1
           A       0.00      0.00      0.00        10
         ABS       0.00      0.00      0.00         2
          AD       0.00      0.00      0.00         4
        ALIM       0.00      0.00      0.00         1
        ANAT       0.00      0.00      0.00         2
        ARCH       0.00      0.00      0.00         1
         ART       0.00      0.00      0.00         0
         BOT       0.00      0.00      0.00       

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344:

############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2  Metrics   #############


{'roc_auc': 0.5, 'hamming_loss': 0.04245709123757904, 'f1': 0.0, 'accuracy_score': 0.0}
              precision    recall  f1-score   support

         (A)       0.00      0.00      0.00         1
        (AD)       0.00      0.00      0.00         0
         (H)       0.00      0.00      0.00         2
        (HD)       0.00      0.00      0.00         1
           A       0.00      0.00      0.00        10
         ABS       0.00      0.00      0.00         2
          AD       0.00      0.00      0.00         4
        ALIM       0.00      0.00      0.00         1
        ANAT       0.00      0.00      0.00         2
        ARCH       0.00      0.00      0.00         1
         ART       0.00      0.00      0.00         0
         BOT       0.00      0.00      0.00         2
        ELEM       0.00      0.00      0.00       

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344:

############# Epoch 3: Training End     #############
############# Epoch 3: Validation Start   #############
############# Epoch 3  Metrics   #############


{'roc_auc': 0.5, 'hamming_loss': 0.04245709123757904, 'f1': 0.0, 'accuracy_score': 0.0}
              precision    recall  f1-score   support

         (A)       0.00      0.00      0.00         1
        (AD)       0.00      0.00      0.00         0
         (H)       0.00      0.00      0.00         2
        (HD)       0.00      0.00      0.00         1
           A       0.00      0.00      0.00        10
         ABS       0.00      0.00      0.00         2
          AD       0.00      0.00      0.00         4
        ALIM       0.00      0.00      0.00         1
        ANAT       0.00      0.00      0.00         2
        ARCH       0.00      0.00      0.00         1
         ART       0.00      0.00      0.00         0
         BOT       0.00      0.00      0.00         2
        ELEM       0.00      0.00      0.00       

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


############# Epoch 31: Training End     #############
############# Epoch 31: Validation Start   #############
############# Epoch 31  Metrics   #############


{'roc_auc': 0.7612404656764352, 'hamming_loss': 0.028906955736224028, 'f1': 0.6097560975609756, 'accuracy_score': 0.5365853658536586}
              precision    recall  f1-score   support

         (A)       0.00      0.00      0.00         1
        (AD)       0.00      0.00      0.00         0
         (H)       0.00      0.00      0.00         2
        (HD)       0.00      0.00      0.00         1
           A       0.78      0.70      0.74        10
         ABS       1.00      0.50      0.67         2
          AD       1.00      1.00      1.00         4
        ALIM       1.00      1.00      1.00         1
        ANAT       0.50      1.00      0.67         2
        ARCH       0.00      0.00      0.00         1
         ART       0.00      0.00      0.00         0
         BOT       1.00      0.50      0.67         2
 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


############# Epoch 32: Training End     #############
############# Epoch 32: Validation Start   #############
############# Epoch 32  Metrics   #############


{'roc_auc': 0.7482436772380568, 'hamming_loss': 0.03432700993676603, 'f1': 0.558139534883721, 'accuracy_score': 0.4878048780487805}
              precision    recall  f1-score   support

         (A)       1.00      1.00      1.00         1
        (AD)       0.00      0.00      0.00         0
         (H)       0.00      0.00      0.00         2
        (HD)       0.00      0.00      0.00         1
           A       0.78      0.70      0.74        10
         ABS       1.00      0.50      0.67         2
          AD       0.50      1.00      0.67         4
        ALIM       1.00      1.00      1.00         1
        ANAT       0.50      1.00      0.67         2
        ARCH       0.00      0.00      0.00         1
         ART       0.00      0.00      0.00         0
         BOT       1.00      0.50      0.67         2
   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [43]:
with open("metrics_final_individual_content.json", "w") as f:
    json.dump(results_individual_determinant_final, f)

# Macro Contents

In [34]:
param_grid_macro_cont = {
                        'max-len' : 128,
                        'learning_rate': 2e-5,
                        'batch_size': 8,
                        'num_train_epochs': 28
                                                }

In [35]:
df_name = dfs[2]
data_model = df_name.split("_")
data_model = "_".join(data_model[1:3]) #getting the name to name the different dictionaries
df = pd.read_csv(df_name)

test_proportion = 0.05
train_size = 0.9

train_df, val_df, df_test, df_labels  = pre_process_data(df, test_proportion, train_size)

In [36]:
print(df_name)

nancy_contents_macro_labels_english.csv


In [37]:
results_macro_content_final = train_final_models(param_grid_macro_cont, train_df, val_df, df_test, df_labels)

############# Epoch 1: Training Start   #############
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1  Metrics   #############


{'roc_auc': 0.5335100945566061, 'hamming_loss': 0.10073710073710074, 'f1': 0.1276595744680851, 'accuracy_score': 0.08108108108108109}
              precision    recall  f1-score   support

  animal_sum       0.75      0.18      0.29        17
   human_sum       0.00      0.00      0.00         9
     abs_sum       0.00      0.00      0.00         2
    food_sum       0.00      0.00      0.00         1
art_arch_sum       0.00      0.00      0.00         1
  nature_sum       0.00      0.00      0.00         4
fragment_sum       0.00      0.00      0.00         2
     geo_sum       0.00      0.00      0.00         0
  object_sum       0.00      0.00      0.00         5
 science_sum       0.00      0.00      0.00         0
 graphic_sum       0.00      0.00      0.00         2

   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2  Metrics   #############


{'roc_auc': 0.6080373115256835, 'hamming_loss': 0.09582309582309582, 'f1': 0.33898305084745767, 'accuracy_score': 0.2702702702702703}
              precision    recall  f1-score   support

  animal_sum       0.62      0.59      0.61        17
   human_sum       0.00      0.00      0.00         9
     abs_sum       0.00      0.00      0.00         2
    food_sum       0.00      0.00      0.00         1
art_arch_sum       0.00      0.00      0.00         1
  nature_sum       0.00      0.00      0.00         4
fragment_sum       0.00      0.00      0.00         2
     geo_sum       0.00      0.00      0.00         0
  object_sum       0.00      0.00      0.00         5
 science_sum       0.00      0.00      0.00         0
 graphic_sum       0.00      0.00      0.00         2

   micro avg       0.62      0.23      0.34        43
   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


############# Epoch 3: Training End     #############
############# Epoch 3: Validation Start   #############
############# Epoch 3  Metrics   #############


{'roc_auc': 0.6586698185535396, 'hamming_loss': 0.07862407862407862, 'f1': 0.46666666666666673, 'accuracy_score': 0.35135135135135137}
              precision    recall  f1-score   support

  animal_sum       0.82      0.82      0.82        17
   human_sum       0.00      0.00      0.00         9
     abs_sum       0.00      0.00      0.00         2
    food_sum       0.00      0.00      0.00         1
art_arch_sum       0.00      0.00      0.00         1
  nature_sum       0.00      0.00      0.00         4
fragment_sum       0.00      0.00      0.00         2
     geo_sum       0.00      0.00      0.00         0
  object_sum       0.00      0.00      0.00         5
 science_sum       0.00      0.00      0.00         0
 graphic_sum       0.00      0.00      0.00         2

   micro avg       0.82      0.33      0.47        43
  

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


############# Epoch 27: Training End     #############
############# Epoch 27: Validation Start   #############
############# Epoch 27  Metrics   #############


{'roc_auc': 0.887107078967544, 'hamming_loss': 0.036855036855036855, 'f1': 0.8192771084337349, 'accuracy_score': 0.7567567567567568}
              precision    recall  f1-score   support

  animal_sum       0.81      1.00      0.89        17
   human_sum       1.00      0.78      0.88         9
     abs_sum       1.00      0.50      0.67         2
    food_sum       1.00      1.00      1.00         1
art_arch_sum       1.00      1.00      1.00         1
  nature_sum       0.75      0.75      0.75         4
fragment_sum       0.00      0.00      0.00         2
     geo_sum       0.00      0.00      0.00         0
  object_sum       1.00      0.40      0.57         5
 science_sum       0.00      0.00      0.00         0
 graphic_sum       1.00      1.00      1.00         2

   micro avg       0.85      0.79      0.82        43
 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


############# Epoch 28: Training End     #############
############# Epoch 28: Validation Start   #############
############# Epoch 28  Metrics   #############


{'roc_auc': 0.8884807053411705, 'hamming_loss': 0.0343980343980344, 'f1': 0.8292682926829267, 'accuracy_score': 0.7837837837837838}
              precision    recall  f1-score   support

  animal_sum       0.81      1.00      0.89        17
   human_sum       1.00      0.78      0.88         9
     abs_sum       1.00      0.50      0.67         2
    food_sum       1.00      1.00      1.00         1
art_arch_sum       1.00      1.00      1.00         1
  nature_sum       0.75      0.75      0.75         4
fragment_sum       0.00      0.00      0.00         2
     geo_sum       0.00      0.00      0.00         0
  object_sum       1.00      0.40      0.57         5
 science_sum       0.00      0.00      0.00         0
 graphic_sum       1.00      1.00      1.00         2

   micro avg       0.87      0.79      0.83        43
  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [38]:
with open("metrics_final_macro_content.json", "w") as f:
    json.dump(results_macro_content_final, f)

# Macro Determinants

In [29]:
param_grid_macro_dets = {
                        'max-len' : 128,
                        'learning_rate': 5e-5,
                        'batch_size': 8,
                        'num_train_epochs': 15
                                                }

In [30]:
df_name = dfs[3]
data_model = df_name.split("_")
data_model = "_".join(data_model[1:3]) #getting the name to name the different dictionaries
df = pd.read_csv(df_name)

test_proportion = 0.05
train_size = 0.9

train_df, val_df, df_test, df_labels  = pre_process_data(df, test_proportion, train_size)

In [31]:
print(df_name)

nancy_determinants_macro_labels_english.csv


In [32]:
results_macro_determinants_final = train_final_models(param_grid_macro_dets, train_df, val_df, df_test, df_labels)

############# Epoch 1: Training Start   #############
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1  Metrics   #############


{'roc_auc': 0.5, 'hamming_loss': 0.24444444444444444, 'f1': 0.0, 'accuracy_score': 0.0}
                  precision    recall  f1-score   support

       color_sum       0.00      0.00      0.00        13
      threat_sum       0.00      0.00      0.00         2
      fading_sum       0.00      0.00      0.00         4
        form_sum       0.00      0.00      0.00        14
kinesthetics_sum       0.00      0.00      0.00        11

       micro avg       0.00      0.00      0.00        44
       macro avg       0.00      0.00      0.00        44
    weighted avg       0.00      0.00      0.00        44
     samples avg       0.00      0.00      0.00        44

############# Epoch 1: Validation End     #############
Epoch: 1 	Avgerage Training Loss: 0.013276 	Average Validati

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2  Metrics   #############


{'roc_auc': 0.5381016042780749, 'hamming_loss': 0.23333333333333334, 'f1': 0.16, 'accuracy_score': 0.05555555555555555}
                  precision    recall  f1-score   support

       color_sum       1.00      0.08      0.14        13
      threat_sum       0.00      0.00      0.00         2
      fading_sum       0.00      0.00      0.00         4
        form_sum       0.67      0.14      0.24        14
kinesthetics_sum       0.50      0.09      0.15        11

       micro avg       0.67      0.09      0.16        44
       macro avg       0.43      0.06      0.11        44
    weighted avg       0.63      0.09      0.16        44
     samples avg       0.10      0.09      0.09        44

############# Epoch 2: Validation End     #############
Epoch: 2 	Avgerage Training Loss: 0.011726 	Average Validation Loss: 0.095585
Vali

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


############# Epoch 3: Training End     #############
############# Epoch 3: Validation Start   #############
############# Epoch 3  Metrics   #############


{'roc_auc': 0.7095588235294118, 'hamming_loss': 0.18333333333333332, 'f1': 0.5714285714285715, 'accuracy_score': 0.4166666666666667}
                  precision    recall  f1-score   support

       color_sum       0.56      0.69      0.62        13
      threat_sum       0.00      0.00      0.00         2
      fading_sum       0.00      0.00      0.00         4
        form_sum       0.64      0.50      0.56        14
kinesthetics_sum       1.00      0.55      0.71        11

       micro avg       0.67      0.50      0.57        44
       macro avg       0.44      0.35      0.38        44
    weighted avg       0.62      0.50      0.54        44
     samples avg       0.56      0.50      0.52        44

############# Epoch 3: Validation End     #############
Epoch: 3 	Avgerage Training Loss: 0.009442 	Average Validation Loss: 

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


############# Epoch 14: Training End     #############
############# Epoch 14: Validation Start   #############
############# Epoch 14  Metrics   #############


{'roc_auc': 0.7931149732620321, 'hamming_loss': 0.15, 'f1': 0.6896551724137931, 'accuracy_score': 0.5277777777777778}
                  precision    recall  f1-score   support

       color_sum       0.60      0.69      0.64        13
      threat_sum       1.00      1.00      1.00         2
      fading_sum       0.50      0.25      0.33         4
        form_sum       0.69      0.64      0.67        14
kinesthetics_sum       0.82      0.82      0.82        11

       micro avg       0.70      0.68      0.69        44
       macro avg       0.72      0.68      0.69        44
    weighted avg       0.69      0.68      0.68        44
     samples avg       0.66      0.66      0.65        44

############# Epoch 14: Validation End     #############
Epoch: 14 	Avgerage Training Loss: 0.000325 	Average Validation Loss: 0.094797
#

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


############# Epoch 15: Training End     #############
############# Epoch 15: Validation Start   #############
############# Epoch 15  Metrics   #############


{'roc_auc': 0.7967914438502672, 'hamming_loss': 0.14444444444444443, 'f1': 0.6976744186046512, 'accuracy_score': 0.5277777777777778}
                  precision    recall  f1-score   support

       color_sum       0.62      0.77      0.69        13
      threat_sum       1.00      0.50      0.67         2
      fading_sum       1.00      0.25      0.40         4
        form_sum       0.69      0.64      0.67        14
kinesthetics_sum       0.82      0.82      0.82        11

       micro avg       0.71      0.68      0.70        44
       macro avg       0.83      0.60      0.65        44
    weighted avg       0.75      0.68      0.69        44
     samples avg       0.68      0.68      0.66        44

############# Epoch 15: Validation End     #############
Epoch: 15 	Avgerage Training Loss: 0.000295 	Average Validation L

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:
with open("metrics_final_macro_determinants.json", "w") as f:
    json.dump(results_macro_determinants_final, f)